# DNN on EPA CDR Data

In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

# Tensorflow imports below...
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GRU
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping

## Read in Data

In [8]:
df = pd.read_csv('/content/drive/MyDrive/ga_data/full_county_dataset.csv')

## Columns to drop to focus on CDR data

In [2]:
non_CDR_columns = ['profile_STATECTY',
 'profile_CNTYNAME',
 'profile_ELEVATION',
 'profile_UID',
 'profile_LAT_DD83',
 'profile_LON_DD83',
 'hab_XBKA',
 'hab_MEDBK_A',
 'hab_XUN',
 'hab_MEDBKUN',
 'hab_XCDENMID',
 'hab_XCDENBK',
 'hab_CONPERCENT',
 'hab_PCT_FA',
 'hab_PCT_DR',
 'hab_PCT_FAST',
 'hab_PCT_SLOW',
 'hab_PCT_POOL',
 'hab_XWIDTH',
 'hab_SDWIDTH',
 'hab_XBKF_W',
 'hab_XBKF_H',
 'hab_XINC_H',
 'hab_SDINC_H',
 'hab_BFWD_RAT',
 'hab_XWXD',
 'hab_XWD_RAT',
 'hab_SDWXD',
 'hab_SDWD_RAT',
 'hab_XDEPTH_CM',
 'hab_SDDEPTH_CM',
 'hab_XFC_ALG',
 'hab_XFC_RCK',
 'hab_XFC_BRS',
 'hab_XFC_LVT',
 'hab_XFC_AQM',
 'hab_XFC_OHV',
 'hab_XFC_HUM',
 'hab_XFC_UCB',
 'hab_XFC_LWD',
 'hab_XFC_NAT',
 'hab_XFC_BIG',
 'hab_XFC_ALL',
 'hab_PCT_SIDE',
 'hab_REACHLEN',
 'hab_W1_HAG',
 'hab_W1_HNOAG',
 'hab_W1_HALL',
 'hab_W1H_BLDG',
 'hab_W1H_LDFL',
 'hab_W1H_LOG',
 'hab_W1H_MINE',
 'hab_W1H_PARK',
 'hab_W1H_PSTR',
 'hab_W1H_PVMT',
 'hab_W1H_PIPE',
 'hab_W1H_ROAD',
 'hab_W1H_CROP',
 'hab_W1H_WALL',
 'hab_C1WM100',
 'hab_C2WM100',
 'hab_C4WM100',
 'hab_V1WM100',
 'hab_V1W_MSQ',
 'hab_V2WM100',
 'hab_V2W_MSQ',
 'hab_V4WM100',
 'hab_V4W_MSQ',
 'hab_PCAN_C',
 'hab_PCAN_D',
 'hab_PCAN_E',
 'hab_PCAN_M',
 'hab_PCAN_N',
 'hab_XCL',
 'hab_XCS',
 'hab_XMW',
 'hab_XMH',
 'hab_XGW',
 'hab_XGH',
 'hab_XGB',
 'hab_XC',
 'hab_XM',
 'hab_XCMW',
 'hab_XCM',
 'hab_XG',
 'hab_XCMGW',
 'hab_XCMG',
 'hab_XPCAN',
 'hab_XPMID',
 'hab_XPMGW',
 'hab_XPCM',
 'hab_XPCMG',
 'hab_XSLOPE',
 'hab_XSLOPE_MAP',
 'hab_XSLOPE_FIELD',
 'hab_PCTCLINOMETER',
 'hab_XBEARING',
 'hab_SINU',
 'hab_LSUB_DMM',
 'hab_LSUBD_SD',
 'hab_LSUB_DMM_NOR',
 'hab_PCT_FN',
 'hab_PCT_GC',
 'hab_PCT_GF',
 'hab_PCT_HP',
 'hab_PCT_OM',
 'hab_PCT_OT',
 'hab_PCT_RC',
 'hab_PCT_SA',
 'hab_PCT_WD',
 'hab_PCT_BIGR',
 'hab_PCT_BDRK',
 'hab_PCT_SAFN',
 'hab_PCT_SFGF',
 'hab_PCT_ORG',
 'hab_XEMBED',
 'hab_XCEMBED',
 'hab_RPXDEP_CM',
 'hab_RPMXDEP_CM',
 'hab_RPGT50',
 'hab_RPGT75',
 'hab_RP100',
 'hab_LTEST',
 'hab_LRBS_TST',
 'hab_LDMB_BW5',
 'hab_LRBS_BW5',
 'hab_LDCBF_G08',
 'hab_LRBS_G08',
 'hab_PCT_SFG',
 'hab_PCT_BH',
 'hab_XSHOR2VG',
 'hab_PCT_OVRB',
 'hab_PCT_GL',
 'hab_C1TM100',
 'hab_C2TM100',
 'hab_C4TM100',
 'hab_PCT_GR',
 'hab_RDIST1',
 'hab_QR1',
 'hab_CVWIDTH',
 'hab_CVWXD',
 'bminv_AMPHNTAX',
 'bminv_AMPHPIND',
 'bminv_AMPHPTAX',
 'bminv_BURRNTAX',
 'bminv_BURRPIND',
 'bminv_BURRPTAX',
 'bminv_CHIRDOM1PIND',
 'bminv_CHIRDOM3PIND',
 'bminv_CHIRDOM5PIND',
 'bminv_CHIRNTAX',
 'bminv_CHIRPIND',
 'bminv_CHIRPTAX',
 'bminv_CLMBNTAX',
 'bminv_CLMBPIND',
 'bminv_CLMBPTAX',
 'bminv_CLNGNTAX',
 'bminv_CLNGPIND',
 'bminv_CLNGPTAX',
 'bminv_COFINTAX',
 'bminv_COFIPIND',
 'bminv_COFIPTAX',
 'bminv_COFITRICNTAX',
 'bminv_COFITRICPIND',
 'bminv_COFITRICPTAX',
 'bminv_COGANTAX',
 'bminv_COGAPIND',
 'bminv_COGAPTAX',
 'bminv_CRUSNTAX',
 'bminv_CRUSPIND',
 'bminv_CRUSPTAX',
 'bminv_DIPTNTAX',
 'bminv_DIPTPIND',
 'bminv_DIPTPTAX',
 'bminv_DOM1PIND',
 'bminv_DOM3PIND',
 'bminv_DOM5PIND',
 'bminv_EPHENTAX',
 'bminv_EPHEPIND',
 'bminv_EPHEPTAX',
 'bminv_EPOTNTAX',
 'bminv_EPOTPIND',
 'bminv_EPOTPTAX',
 'bminv_EPT_NTAX',
 'bminv_EPT_PIND',
 'bminv_EPT_PTAX',
 'bminv_FACLNTAX',
 'bminv_FACLPIND',
 'bminv_FACLPTAX',
 'bminv_HEMINTAX',
 'bminv_HEMIPIND',
 'bminv_HEMIPTAX',
 'bminv_HPRIME',
 'bminv_INTLNTAX',
 'bminv_INTLPIND',
 'bminv_INTLPTAX',
 'bminv_MITENTAX',
 'bminv_MITEPIND',
 'bminv_MITEPTAX',
 'bminv_MOLLNTAX',
 'bminv_MOLLPIND',
 'bminv_MOLLPTAX',
 'bminv_NOINNTAX',
 'bminv_NOINPIND',
 'bminv_NOINPTAX',
 'bminv_NTOLNTAX',
 'bminv_NTOLPIND',
 'bminv_NTOLPTAX',
 'bminv_ODONNTAX',
 'bminv_ODONPIND',
 'bminv_ODONPTAX',
 'bminv_OLLENTAX',
 'bminv_OLLEPIND',
 'bminv_OLLEPTAX',
 'bminv_ORTHCHIRPIND',
 'bminv_ORTHNTAX',
 'bminv_ORTHPIND',
 'bminv_ORTHPTAX',
 'bminv_PLECNTAX',
 'bminv_PLECPIND',
 'bminv_PLECPTAX',
 'bminv_PREDNTAX',
 'bminv_PREDPIND',
 'bminv_PREDPTAX',
 'bminv_SCRPNTAX',
 'bminv_SCRPPIND',
 'bminv_SCRPPTAX',
 'bminv_SHRDNTAX',
 'bminv_SHRDPIND',
 'bminv_SHRDPTAX',
 'bminv_SPWLNTAX',
 'bminv_SPWLPIND',
 'bminv_SPWLPTAX',
 'bminv_STOLNTAX',
 'bminv_STOLPIND',
 'bminv_STOLPTAX',
 'bminv_SWIMNTAX',
 'bminv_SWIMPIND',
 'bminv_SWIMPTAX',
 'bminv_TANYNTAX',
 'bminv_TANYPIND',
 'bminv_TANYPTAX',
 'bminv_TL01NTAX',
 'bminv_TL01PIND',
 'bminv_TL01PTAX',
 'bminv_TL23NTAX',
 'bminv_TL23PIND',
 'bminv_TL23PTAX',
 'bminv_TL45NTAX',
 'bminv_TL45PIND',
 'bminv_TL45PTAX',
 'bminv_TL67NTAX',
 'bminv_TL67PIND',
 'bminv_TL67PTAX',
 'bminv_TOLRNTAX',
 'bminv_TOLRPIND',
 'bminv_TOLRPTAX',
 'bminv_TOTLNIND',
 'bminv_TOTLNTAX',
 'bminv_TRICNTAX',
 'bminv_TRICPIND',
 'bminv_TRICPTAX',
 'bminv_TUBINAIDNTAX',
 'bminv_TUBINAIDPIND',
 'bminv_TUBINAIDPTAX',
 'bminv_WTD_TV']

In [9]:
#Demographic columns
demo_string = "name	fips	age_over_65	state	pop2017	poverty	homeownership	multi_unit	unemployment_rate	metro	median_edu	median_hh_income	smoking_ban	incidence_rate_per_100k	recent_trend_cat	5yr_trend"
demo_columns = demo_string.split()

In [10]:
df_CDR = df.drop(columns=non_CDR_columns).copy()

In [12]:
df_CDR.dropna(inplace=True)

In [13]:
df_CDR['recent_trend_cat'].value_counts()

stable     1145
rising      253
falling      38
Name: recent_trend_cat, dtype: int64

In [23]:
y = df_CDR['recent_trend_cat'].copy()

In [24]:
y.value_counts(normalize=True).max()

0.7973537604456824

In [15]:
df_CDR.dtypes

name                                                          object
fips                                                           int64
age_over_65                                                  float64
state                                                         object
pop2017                                                      float64
                                                              ...   
4897    Soybean oil, Me ester, polymd., oxidized             float64
5023    Dodecane, 1-(methylthio)-, manuf. of, by-produ...    float64
5808    Naphtha (petroleum), heavy catalytic cracked, ...    float64
8227    Fatty acids, tall-oil, maleated, sodium salts        float64
8228    Fatty acids, tall-oil, reaction products with ...    float64
Length: 6532, dtype: object

In [16]:
X = df_CDR.select_dtypes(exclude=['object'])


In [34]:
oh = OneHotEncoder(
    sparse=False,
    dtype=int,
    categories="auto"
)

y = oh.fit_transform(y.values.reshape(-1, 1))

In [18]:
df_CDR.describe().max().sort_values(ascending=False)

pop2017                                                       10163507.0
median_hh_income                                                117515.0
fips                                                             56045.0
14698    2-Propenoic acid, 2-methyl-, 2-hydroxyethyl es...        1436.0
14863    Phosphorodithioic acid, O,O-bis(2-methylpropyl...        1436.0
                                                                 ...    
10819    Octanoic acid, sodiumsalt (1:1)                          1436.0
10816    Benzoic acid, 3,6-dichloro-2-methoxy-, sodium ...        1436.0
10804    1,4-Benzenediol, 2-(1,1-dimethylethyl)-                  1436.0
10796    1-Propanaminium, N,N,N-tripropyl-, bromide (1:1)         1436.0
8228    Fatty acids, tall-oil, reaction products with ...         1436.0
Length: 6529, dtype: float64

In [19]:
df_CDR.shape

(1436, 6532)

In [22]:
sc = StandardScaler()
X_sc = sc.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_sc, y, stratify= y, test_size=0.2, random_state=4)

model = Sequential()

model.add(Dense(1000, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(500, activation='relu', kernel_regularizer=l2(0.002)))
model.add(Dropout(.05))
model.add(Dense(256, activation='relu', kernel_regularizer=l2(0.002)))
model.add(Dropout(.1))
model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.002)))
model.add(Dropout(.3))
model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.002)))
model.add(Dropout(.5))
model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.002)))
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.002)))
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['mae', 'acc'])
early_stop = EarlyStopping(monitor='val_loss', min_delta=0.1, patience=5, verbose=1, mode='auto')

history = model.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    epochs=20,
    batch_size=32,
    callbacks=[early_stop]

)

Epoch 1/20
36/36 [==============================] - 3s 14ms/step - loss: 2.8801 - mae: 0.2521 - acc: 0.7962 - val_loss: 2.1397 - val_mae: 0.2632 - val_acc: 0.7986
Epoch 2/20
36/36 [==============================] - 0s 6ms/step - loss: 1.7372 - mae: 0.2030 - acc: 0.7970 - val_loss: 1.4500 - val_mae: 0.2063 - val_acc: 0.7986
Epoch 3/20
36/36 [==============================] - 0s 6ms/step - loss: 1.2380 - mae: 0.1872 - acc: 0.7970 - val_loss: 1.1370 - val_mae: 0.2217 - val_acc: 0.7986
Epoch 4/20
36/36 [==============================] - 0s 8ms/step - loss: 0.9837 - mae: 0.1802 - acc: 0.7970 - val_loss: 0.9531 - val_mae: 0.1937 - val_acc: 0.7986
Epoch 5/20
36/36 [==============================] - 0s 8ms/step - loss: 0.8126 - mae: 0.1644 - acc: 0.7970 - val_loss: 0.9117 - val_mae: 0.1481 - val_acc: 0.7986
Epoch 6/20
36/36 [==============================] - 0s 7ms/step - loss: 0.6871 - mae: 0.1488 - acc: 0.8188 - val_loss: 0.7482 - val_mae: 0.1535 - val_acc: 0.8194
Epoch 7/20
36/36 [=========

In [25]:
preds = model.predict(X_sc)

45/45 [==============================] - 0s 2ms/step


In [26]:
preds_classes =np.argmax(preds,axis=1)

In [37]:
preds_classes

array([2, 1, 2, ..., 1, 1, 1])

While we attempted to make predictions in 3 classes (Rising, Stable, and Falling), our model only predicted Rising and Stable, likely because there are so few Falling counties. 

In [27]:
np.unique(preds_classes, return_counts=True)

(array([1, 2]), array([ 279, 1157]))

In [31]:
with open('/content/drive/MyDrive/ga_data/cdr_nih_20221121.pkl', 'wb') as pickle_out:
    pickle_out = pickle.dump(model, pickle_out)

In [ ]:
with open('/content/drive/MyDrive/ga_data/cdr_nih_20221121.pkl', 'rb') as pickle_out:
    model = pickle.load(pickle_out)